## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-13-01-15-14 +0000,nyt,Trump’s Plans for Venezuelan Oil Run Headlong ...,https://www.nytimes.com/2026/01/12/us/politics...
1,2026-01-13-01-14-16 +0000,bbc,How Australian festival imploded after axing P...,https://www.bbc.com/news/articles/cgm4jkwz2z8o...
2,2026-01-13-01-14-00 +0000,wsj,Oil Rises Amid Middle East Tensions,https://www.wsj.com/finance/commodities-future...
3,2026-01-13-01-08-23 +0000,nyt,Trump Has Another Justification for the Shooti...,https://www.nytimes.com/2026/01/12/us/politics...
4,2026-01-13-01-04-58 +0000,nyt,Top Fed Official Conveys Little Urgency for Im...,https://www.nytimes.com/2026/01/12/business/fe...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2520/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,61
89,iran,33
25,fed,21
79,powell,20
285,new,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
213,2026-01-12-16-36-01 +0000,nypost,GOP Sen. Thom Tillis says he will block any of...,https://nypost.com/2026/01/12/business/gop-sen...,150
280,2026-01-12-10-30-00 +0000,wsj,"For years, Fed Chair Jerome Powell responded t...",https://www.wsj.com/economy/central-banking/fo...,148
85,2026-01-12-22-04-31 +0000,cbc,Bank of Canada governor defends U.S. Fed's Pow...,https://www.cbc.ca/news/politics/bank-of-canad...,134
308,2026-01-12-05-36-07 +0000,bbc,Trump mulls 'very strong' military options as ...,https://www.bbc.com/news/articles/czdqy3rmp78o...,133
318,2026-01-12-03-13-22 +0000,cbc,Former Federal Reserve leaders slam Trump admi...,https://www.cbc.ca/news/world/jerome-powell-fe...,133


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
213,150,2026-01-12-16-36-01 +0000,nypost,GOP Sen. Thom Tillis says he will block any of...,https://nypost.com/2026/01/12/business/gop-sen...
42,82,2026-01-12-23-21-39 +0000,nypost,Iran to hang 26-year-old in alleged first exec...,https://nypost.com/2026/01/12/world-news/iran-...
90,62,2026-01-12-21-55-00 +0000,wsj,The Justice Department’s investigation of Fede...,https://www.wsj.com/economy/central-banking/th...
241,58,2026-01-12-13-24-31 +0000,cbc,Swiss court keeps bar owner in custody after d...,https://www.cbc.ca/news/world/swiss-court-bar-...
262,46,2026-01-12-11-50-09 +0000,nypost,Gut-wrenching footage shows grieving Iranian f...,https://nypost.com/2026/01/12/world-news/foota...
54,44,2026-01-12-23-02-33 +0000,bbc,Minnesota sues Trump administration to block s...,https://www.bbc.com/news/articles/c394xywlj8jo...
168,44,2026-01-12-18-50-00 +0000,nypost,UK probes X over Grok AI chatbot’s sexualized ...,https://nypost.com/2026/01/12/business/uk-prob...
58,42,2026-01-12-22-50-22 +0000,nypost,Teen charged in fatal NYC stabbing of 17-year-...,https://nypost.com/2026/01/12/us-news/teen-cha...
38,41,2026-01-12-23-31-24 +0000,nypost,"Hegseth tells defense contractors to ‘step up,...",https://nypost.com/2026/01/12/us-news/hegseth-...
191,36,2026-01-12-17-43-12 +0000,nypost,Tiny deer goes head-to-head with a 2-ton rhino...,https://nypost.com/2026/01/12/world-news/tiny-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
